In [1]:

import json
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

connection_parameters = json.load(open('../connection.json'))
session = Session.builder.configs(connection_parameters).create()



###  Setup 

This will populate the ALL_CLAIMS_RAW table from a CSV file and doesn't need  SPCS as we are not running Whisper model in the container. 

 
 Run the below commands in snowsight as <b>ACCOUNTADMIN</b>

You have to execute the below commands only once for all the three containers you will be creating as part of the solution. You will be creating the other pools while crearting other SPCS services.

``` sql

USE ROLE ACCOUNTADMIN;

CREATE ROLE SPCS_PSE_ROLE;

CREATE DATABASE IF NOT EXISTS LLMDemo;
GRANT OWNERSHIP ON DATABASE LLMDemo TO ROLE SPCS_PSE_ROLE COPY CURRENT GRANTS;
GRANT OWNERSHIP ON ALL SCHEMAS IN DATABASE LLMDemo  TO ROLE SPCS_PSE_ROLE COPY CURRENT GRANTS;

CREATE OR REPLACE WAREHOUSE small_warehouse WITH
  WAREHOUSE_SIZE='X-SMALL';
GRANT USAGE ON WAREHOUSE small_warehouse TO ROLE SPCS_PSE_ROLE;

CREATE SECURITY INTEGRATION IF NOT EXISTS snowservices_ingress_oauth
  TYPE=oauth
  OAUTH_CLIENT=snowservices_ingress
  ENABLED=true;

GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE SPCS_PSE_ROLE;

GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE SPCS_PSE_ROLE;

GRANT ROLE SPCS_PSE_ROLE TO USER <user_name>;

USE ROLE SPCS_PSE_ROLE;
USE DATABASE LLMDemo;
USE WAREHOUSE small_warehouse;
USE SCHEMA PUBLIC;


```


In [19]:
# Creating the Table to load the Audio file raw text along with duration and other attributes

# Duration is in seconds

session.sql('''
    CREATE or REPLACE TABLE ALL_CLAIMS_RAW (
	DATETIME DATE,
	AUDIOFILE VARCHAR(16777216),
	CONVERSATION VARCHAR(16777216),
	PRESIGNED_URL_PATH VARCHAR(16777216),
	DURATION FLOAT NOT NULL
)''').collect()


[Row(status='Table DEMO_ALL_CLAIMS_RAW successfully created.')]

#### Loading Data into the ALL_CLAIMS_RAW Table

Since we don't have lot of audio files from insurance industry, we will be loading sample data into the Raw table which has the raw conversation from the insurance industry.

In [17]:
_ = session.file.put("./Sample_Audio_Text.csv", "@CSV_FILES", auto_compress=False)

sp_df=session.read.options({"INFER_SCHEMA":True,"PARSE_HEADER":True,"FIELD_OPTIONALLY_ENCLOSED_BY":'"'}).csv('@CSV_FILES/Sample_Audio_Text.csv')

# sp_df = session.read.option("INFER_SCHEMA", True).option("PARSE_HEADER", True).option("FIELD_OPTIONALLY_ENCLOSED_BY",'"').csv("@CSV_FILES/Sample_Audio_Text.csv")

In [20]:
sp_df.write.mode("overwrite").save_as_table("ALL_CLAIMS_RAW")

In [21]:
session.table('ALL_CLAIMS_RAW').to_pandas()

,DATETIME,AUDIOFILE,CONVERSATION,PRESIGNED_URL_PATH,DURATION
0,2023-11-11,audiofile1.mp3,"Hello, this is Emily from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,218.254271
1,2023-11-11,audiofile2.mp3,"Hello, this is James from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,197.705486
2,2023-11-11,audiofile3.mp3,"Hello, this is Sarah from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,75.172382
3,2023-11-11,audiofile4.mp3,"Hello, this is Kevin from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,224.291618
4,2023-11-15,audiofile5.mp3,"Hello, this is Olivia from AutoAssure Insuranc...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,174.649442
...,...,...,...,...,...
91,2024-01-05,audiofile92.mp3,"Good morning, this is Sara at AutoAssure Insur...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,102.048162
92,2024-01-05,audiofile93.mp3,"Hello, I'm Michael with AutoAssure Insurance, ...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,71.842008
93,2024-01-10,audiofile95.wav,"Welcome to AutoAssure Insurance, this is Josh....",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,84.340859
94,2024-01-10,audiofile97.wav,"Hi, this is Ethan from AutoAssure Insurance. W...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,195.237454


#### Loading Data into the Final Tables

Run the below notebook which will populate data into multiple tables required for the Streamlit App

[Audio Analytics Notebook](./AudioAnalytics.ipynb)
